<a href="https://colab.research.google.com/github/SteveWaite/DataRays/blob/main/2simulaton_bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import PIL
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvas


In [ ]:
red = [204,0,0]
orange = [255,153,51]
yellow = [255,255,102]
lime = [153,255,51]
green = [51,255,51]
seagrn = [51,255,153]
turquoise = [51,255,255]
blue =[51,153,255]
blupurp = [51,51,255]
purple = [153,51,255]

Bidge assumptions
-no passing on bridge
-trucks can only be in outside lane
-entire lane has same speed
-entire lane has same acc
-bridge length is 400m
-bridge deck is divided into 100 length segments
-inside lane speed is on average 25m/sec
-outside lane speed is on average 22.22 m/sec
-bidge deck is divided into spaces of length 4m and one lane width
-lane speed and acceleration vary with each time step
-a new vehicle enters the bridge every time step
-there are 3 vehicle types(passenger car, delivery truck, truck-trailer semi)
-truck-trailers a least common and passenger and delivery trucks enter twice as often as truck-trailers
-vehicle weight is set specifically for each vehicle and remains constant over entire passage on bridge
-time steps correspond to time needed to move 1 position segment at 25m/sec
-position of a vehicle varies according to lane speed only
-lane acceleration is calculated at each time step by dividing the difference in speed of actual segment and previous segment by 0.16 sec (time to pass over segment at 25m/s)
-time of passage over segment is fixed at 0.16 secs regardless of speed variations
-no passing is possible in the simulation
-only one vehicle per segment is possible
-all vehicles are assumed to fit into a single segment
-as vehicles move over the bridge according to lane speed a tilt sensor measures the variation of pitch, roll and yaw
- pitch is influenced by cumulative effects of vehicle weight, the distance of vehicles from the sensor and the speed of the vehicle (same speed for whole lane)
- roll is influenced by cummulative effects of vehicle weight and distance from bridge centerline (inside lane or outside lane)
-yaw is influenced by cumulative effects of vehicle weight, distance from bridge center and lane acceleration
-the relationship between each of the sensor variables and the traffic loads is fixed by an equation that takes into account the influence of the variation of all loads
Graphic presentation





In [ ]:
num_segments = int(25)
num_lanes = 4
veh_types = [1,2,1,2,0]
veh_lanes = [0,1,2,3]
veh_names = {}
cycles = int(10000)
image_matrix = np.ones(((cycles,100,4)))*255
vert_offset = int(15)
horiz_offset = int(56)
e = 0
s = list(np.linspace(-np.pi/2,3/2*np.pi,num_segments))
lane_sp = np.zeros(4)
last_lane_sp = np.zeros(4)
lane_acc = np.zeros(4)
value_limit_check = np.zeros((4,cycles))
vehs_lane_00 = {} 
vehs_lane_01 = {}
vehs_lane_02 = {}
vehs_lane_03 = {}

start_rec = int(300)

# t = time step (25 because there are 25 time steps per graph)
# each time step has same speed & acceleration for all vehicles in a lane

d = 0
for t in range (cycles):
  # speed for each lane is set separately every time step
  # speed for inside lanes are mostly higher than outside lanes
  # all vehicles in a lane have the same speed so that there is not bunching
  lane_sp[0] = 22.22 +  tf.random.normal([1],0,2)
  if lane_sp[0] > 30:
    lane_sp[0] = 30
  if lane_sp[0] < 20:
    lane_sp[0] = 20
  lane_sp[1] = 25 +  tf.random.normal([1],0,2)
  if lane_sp[1] > 30:
    lane_sp[1] = 30
  if lane_sp[1] < 20:
    lane_sp[1] = 20
  lane_sp[2] = 25 +  tf.random.normal([1],0,2)
  if lane_sp[2] > 30:
    lane_sp[2] = 30
  if lane_sp[2] < 20:
    lane_sp[2] = 20
  lane_sp[3] = 22.22 +  tf.random.normal([1],0,2)
  if lane_sp[3] > 30:
    lane_sp[3] = 30
  if lane_sp[3] < 20:
    lane_sp[3] = 20

  # lane acceleration is set at each time step 
  # all vehicles in a lane have same acceleration
  # acceleration is calculated as the change in speed divided by the length
  # of time each time step represents

  lane_acc =  (lane_sp - last_lane_sp) / 0.64 / 5
  for i in range (len(lane_acc)):
    if lane_acc[i] < -2:
      lane_acc[i] = -2
    if lane_acc[i] > 2:
      lane_acc[i] = 2 

  # because the first time step cannot have a speed difference acc is zero
  if t ==0:
    lane_acc = np.zeros(4)

  #print('lane_sp = ',lane_sp)
  #print('last_lane_sp = ', last_lane_sp)

  #last_lane_sp is the speed of the previous time step
  last_lane_sp[:] = lane_sp[:]

  # the vehicle type to be created is set for each time step
  # there are three vehicle types (light, medium and heavy)
  # one type is chosen each time step and introduced

  veh_type_shuffled = list(tf.random.shuffle(veh_types))
  veh_type = veh_type_shuffled[0]
  
  # according to vehicle type there are 3 vehicle weight ranges
  # vehicle weight varies around an average value for each vehicle type

  if veh_type == 0:
    veh_weight = tf.random.normal([1],0.9,0.2)
    veh_weight = np.round(veh_weight,decimals=2)
  if veh_type == 1:
    veh_weight = tf.random.normal([1],2,0.4)
    veh_weight = np.round(veh_weight,decimals=2)
  if veh_type == 2:
    veh_weight = tf.random.normal([1],6,0.5)
    veh_weight = np.round(veh_weight,decimals=2)

  # the bridge has four lanes
  # light and medium weight vehicles can use any lane 
  if veh_type != 2:
    veh_lane_shuffled = list(tf.random.shuffle(veh_lanes))
    newveh_lane = veh_lane_shuffled[0]
  # heavy vehicles can only use outside lanes
  else:
    veh_lane_shuffled = list(tf.random.shuffle([0,3]))
    newveh_lane = veh_lane_shuffled[0]

  # to be used as colors in rgb image values must be between 0 and 255
  pos_sp = np.zeros(4)
  pos_acc = np.zeros(4)

  # pos_sp is lane speed value for .jpeg image (color ranges are pre-chosen)

  pos_sp[0] = int((lane_sp[0]-20)/(30-20)*100+150)  #(red 150-250)
  pos_sp[1] = int((lane_sp[1]-20)/(30-20)*100+150)
  pos_sp[2] = int((lane_sp[2]-20)/(30-20)*100+150)
  pos_sp[3] = int((lane_sp[3]-20)/(30-20)*100+150)

  # pos_acc is lane acceleration for .jpeg image
  pos_acc[0] =  int((lane_acc[0]-(-2))/(2-(-2))*100) #(green 0-100) 
  pos_acc[1] =  int((lane_acc[1]-(-2))/(2-(-2))*100)
  pos_acc[2] =  int((lane_acc[2]-(-2))/(2-(-2))*100)
  pos_acc[3] =  int((lane_acc[3]-(-2))/(2-(-2))*100)

  # pos_veh_weight is vehicle weight for .jpeg image
  # max veh_weight = 7 tons and min veh_weight = 0.6 tons
  pos_veh_weight = int(150 + (veh_weight - 0.6)/(7-0.6)*100)  #(blue 150-250)

  # chosen lane for time step new vehicle creation
  veh_lanes_shuffled = list(tf.random.shuffle(veh_lanes))
  newveh_lane = veh_lanes_shuffled[0] 

  # create one new vehicle to be introduced onto the bridge each time step
  # new vehicle is named with the time step
  # the vehicle name never changes for a given vehicle which moves over the bridge
  veh_name = f'V{t}'
  #print('t =',t)
  print('veh_name =', veh_name)
  #print('pos_veh_weight =', pos_veh_weight)
  
  # vehicle color is determined by 3 values between 0 and 255
  # vehicle speed and acceleration change with each t but weight is always same for given vehicle
  # the last value is the vehicle position on the bridge (each lane 100 positions)
  veh_color = np.zeros(4)


  if newveh_lane == 0:
    veh_color[0] = pos_sp[0]
    veh_color[1] = pos_acc[0]
    veh_color[2] = pos_veh_weight
    veh_color[3] = 0 
    vehs_lane_00[veh_name] = veh_color

  if newveh_lane == 1:
    veh_color[0] = pos_sp[1]
    veh_color[1] = pos_acc[1]
    veh_color[2] = pos_veh_weight
    veh_color[3] = 25 
    vehs_lane_01[veh_name] = veh_color

  if newveh_lane == 2:
    veh_color[0] = pos_sp[2]
    veh_color[1] = pos_acc[2]
    veh_color[2] = pos_veh_weight
    veh_color[3] = 74
    vehs_lane_02[veh_name] = veh_color

  if newveh_lane == 3:
    veh_color[0] = pos_sp[3]
    veh_color[1] = pos_acc[3]
    veh_color[2] = pos_veh_weight
    veh_color[3] = 99    
    vehs_lane_03[veh_name] = veh_color
  
  # move vehicles  
  
  for l in range (4):
    if l == 0:
      veh_speed = pos_sp[l]
      move0 = veh_speed * 0.64/16/2
      #print('move0 = ', move0)
      if len(vehs_lane_00) != 0:
        #print('number of veh in lane 0 =',len(vehs_lane_00))        
        for k, v in list(vehs_lane_00.items()):
          if v[-1] + move0 > 24:
            del vehs_lane_00[k]
            print(f'lane0 del{k}')
          else:
            v[-1] += move0
            #print(f'lane0 position of{k} = ', v[-1])                                
        for item in vehs_lane_00.items():
          image_matrix_y =  int(item[1][-1]) 
          image_matrix[t, image_matrix_y,:] = [int(item[1][0]),int(item[1][1]),int(item[1][2]),int(item[1][3])]

    if l == 1:
      veh_speed = pos_sp[l]
      move1 = veh_speed * 0.64/16/2
      #print('move1 = ', move1)
      if len(vehs_lane_01) != 0:
        #print('number of veh in lane 1 =',len(vehs_lane_01))        
        for k, v in list(vehs_lane_01.items()):
          if v[-1] + move1 > 49:
            del vehs_lane_01[k]
            print(f'lane1 del{k}')
          else:
            v[-1] += move1
            #print(f'lane1 position of{k} = ', v[-1])                                
        for item in vehs_lane_01.items():
          image_matrix_y =  int(item[1][-1]) 
          image_matrix[t, image_matrix_y,:] = [int(item[1][0]),int(item[1][1]),int(item[1][2]),int(item[1][3])]


    if l == 2:
      veh_speed = pos_sp[l]
      move2 = veh_speed * 0.64 * (-1)/16/2
      #print('move2 = ', move2)
      if len(vehs_lane_02) != 0:
        #print('number of veh in lane 2 =',len(vehs_lane_02))
        for k, v in list(vehs_lane_02.items()):
          if v[-1] + move2 <= 50:
            del vehs_lane_02[k]
            print(f'lane2 del{k}')
          else:
            v[-1] += move2
            #print(f'lane2 position of{k} = ', v[-1])                               
        for item in vehs_lane_02.items():
          image_matrix_y =  int(item[1][-1]) 
          image_matrix[t, image_matrix_y,:] = [int(item[1][0]),int(item[1][1]),int(item[1][2]),int(item[1][3])]

    if l == 3:
      veh_speed = pos_sp[l]
      move3 = veh_speed * 0.64 * (-1)/16/2
      #print('move3 = ', move3)
      if len(vehs_lane_03) != 0:
        #print('number of veh in lane 3 =',len(vehs_lane_03))
        for k, v in list(vehs_lane_03.items()):
          if v[-1] + move3 <= 75:
            del vehs_lane_03[k]
            print(f'lane3 del{k}')
          else:
            v[-1] += move3
            #print(f'lane3 position of{k} = ', v[-1])                                
        for item in vehs_lane_03.items():
          image_matrix_y =  int(item[1][-1]) 
          image_matrix[t, image_matrix_y,:] = [int(item[1][0]),int(item[1][1]),int(item[1][2]),int(item[1][3])]                               

              # vehicle weight stays always same so no update

  speed_matrix = np.zeros((num_lanes,25))
  speed_matrix[0,:] = image_matrix[t,:25,0]
  speed_matrix[1,:] = image_matrix[t,25:50,0]
  speed_matrix[2,:] = image_matrix[t,50:75,0]
  speed_matrix[3,:] = image_matrix[t,75:100,0]

  acc_matrix = np.zeros((num_lanes,25))
  acc_matrix[0,:] = image_matrix[t,:25,1]
  acc_matrix[1,:] = image_matrix[t,25:50,1]
  acc_matrix[2,:] = image_matrix[t,50:75,1]
  acc_matrix[3,:] = image_matrix[t,75:100,1]

  weight_matrix = np.zeros((num_lanes,25))
  weight_matrix[0,:] = image_matrix[t,:25,2]
  weight_matrix[1,:] = image_matrix[t,25:50,2]
  weight_matrix[2,:] = image_matrix[t,50:75,2]
  weight_matrix[3,:] = image_matrix[t,75:100,2]

  pitch_amp = 0
  for l in range(num_lanes):
    for j in range(25): 
      factor = (np.sin(s[j])+1)/2
      if l == 0 or l ==1:
        pitch_amp += weight_matrix[l,j]*factor*speed_matrix[l,j]
      else:
        pitch_amp -= weight_matrix[l,j]*factor*speed_matrix[l,j]

  roll_amp =0
  for l in range(num_lanes):
    for j in range(25): 
      factor = (np.sin(s[j])+1)/2
      if l == 1 or l ==2:
        if l == 1:
          roll_amp += weight_matrix[l,j]*factor
        else:
          roll_amp -= weight_matrix[l,j]*factor
      else:
        if l == 0:
          roll_amp += weight_matrix[l,j]*factor*2
        else:
          roll_amp -= weight_matrix[l,j]*factor*2 

  yaw_amp =0
  for l in range(num_lanes):
    for j in range(25): 
      factor = (np.sin(s[j])+1)/2
      if l == 1 or l ==2:
        if l == 1:
          yaw_amp += weight_matrix[l,j]*factor*acc_matrix[l,j]
        else:
          yaw_amp -= weight_matrix[l,j]*factor*acc_matrix[l,j]
      else:
        if l == 0:
          yaw_amp += weight_matrix[l,j]*factor*acc_matrix[l,j]*2
        else:
          yaw_amp -= weight_matrix[l,j]*factor*acc_matrix[l,j]*2 

  value_limit_check[0,t] = pitch_amp*2/1.5
  value_limit_check[1,t] = roll_amp*2/1.5
  value_limit_check[2,t] = yaw_amp/2/1.5
  value_limit_check[3,t] = t
  """
  print('newveh_lane = ', newveh_lane)
  print('vehs_lane_00 = ', vehs_lane_00)
  print('vehs_lane_01 = ', vehs_lane_01)
  print('vehs_lane_02 = ', vehs_lane_02)
  print('vehs_lane_03 = ', vehs_lane_03)
  print('pos_sp = ', pos_sp)
  print('pos_acc = ', pos_acc) 
  """ 

  if t >= start_rec:
    """
    print('newveh_lane = ', newveh_lane)
    print('vehs_lane_00 = ', vehs_lane_00)
    print('vehs_lane_01 = ', vehs_lane_01)
    print('vehs_lane_02 = ', vehs_lane_02)
    print('vehs_lane_03 = ', vehs_lane_03)
    print('pos_sp = ', pos_sp)
    print('pos_acc = ', pos_acc)
    print('pos_veh_weight = ',pos_veh_weight)
    """

    grid_image = image_matrix[int(t-25):t,:,:4]
    grid_final = np.zeros(((100,25,3)))
    grid_final[75:100,:,:] = grid_image[:,:25,:-1]
    grid_final[50:75,:,:] = grid_image[:,25:50,:-1]
    grid_final[25:50,:,:] = grid_image[:,50:75,:-1]
    grid_final[:25,:,:] = grid_image[:,75:100,:-1]

    magv = int(4)
    magh = int(16)

    Blow_up = np.zeros(((100*magv,25*magh,3)))

    patch = np.zeros(((magv,magh,3)))
    for x in range (100):
      for y in range(25):
        color = grid_final[x,y,:]
        for j in range (magv):
          for k in range (magh):
            patch[j,k,:] = color
        coordxbegin = x*magv
        coordxend = (x*magv)+magv
        coordybegin = y*magh
        coordyend = (y*magh)+magh
        Blow_up[coordxbegin:coordxend,coordybegin:coordyend,:] = patch


    background = np.asarray(Blow_up, dtype=np.dtype('uint8'))

    img = Image.fromarray(background, 'RGB')

    img.save('/content/drive/MyDrive/bridge_files2/Blow_up.png', dpi=(512, 512))    
    img.close()

    time = np.array(value_limit_check[3,t-25:t])
    pitch = value_limit_check[0,t-25:t]
    roll = (value_limit_check[1,t-25:t])*200
    yaw = value_limit_check[2,t-25:t]

    fig, ax = plt.subplots() 
    ax.set_xlim(t-25,t)
    ax.set_ylim(-300000,300000)    
    ax.plot(time,pitch,color="red",linewidth=2)
    ax.plot(time,roll,color="blue",linewidth=2)
    ax.plot(time,yaw,color="green",linewidth=2)
    plt.axis('off')
    plt.savefig('/content/drive/MyDrive/bridge_files2/graph.png', transparent=True)    
    #plt.savefig(f'/content/drive/MyDrive/bridge_files2/bridge_files2{d}/A{t}.jpg')
    plt.close(fig)

    im1 = Image.open(r"/content/drive/MyDrive/bridge_files2/Blow_up.png")
    im2 = Image.open(r"/content/drive/MyDrive/bridge_files2/graph.png")
    im1.paste(im2, (0,0), mask = im2)
    newsize = (512, 512)
    im1 = im1.resize(newsize)
    im1.save(f'/content/drive/MyDrive/bridge_files2/bridge_files2{d}/H{t}.jpg') 
    im1.close()
    im2.close()
    
    d += 1
    if d == 8:
      d = 0
    

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
lane3 delV7493
veh_name = V7501
lane0 delV7494
lane2 delV7495
lane2 delV7496
veh_name = V7502
lane2 delV7497
veh_name = V7503
veh_name = V7504
veh_name = V7505
lane0 delV7498
lane2 delV7500
veh_name = V7506
lane0 delV7499
veh_name = V7507
lane1 delV7502
veh_name = V7508
lane3 delV7501
veh_name = V7509
lane2 delV7503
veh_name = V7510
lane2 delV7504
veh_name = V7511
lane1 delV7506
veh_name = V7512
lane3 delV7505
veh_name = V7513
veh_name = V7514
lane1 delV7509
lane3 delV7507
lane3 delV7508
veh_name = V7515
veh_name = V7516
veh_name = V7517
lane0 delV7510
lane2 delV7512
veh_name = V7518
lane0 delV7511
veh_name = V7519
lane2 delV7513
veh_name = V7520
lane3 delV7514
veh_name = V7521
lane0 delV7515
lane1 delV7516
veh_name = V7522
veh_name = V7523
lane1 delV7517
veh_name = V7524
lane2 delV7518
veh_name = V7525
lane1 delV7520
lane3 delV7519
veh_name = V7526
lane1 delV7521
veh_name = V7527
veh_name = V7528
lane0 delV7

In [ ]:
check = Image.open(r"/content/drive/MyDrive/bridge_files2/bidge_files20/A1004.jpg")
width, height = check.size
print(width,"x",height)

FileNotFoundError: ignored